In [ ]:
# Import packages
import numpy as np
from numpy import transpose as T
from numpy import genfromtxt as gftxt
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import glob
import pandas as pd
import os
import cartopy
import cartopy.crs as ccrs
import scipy
from scipy import stats

# # # #
# Define the custom Lambert Azimuthal Equal Area projection
map_projection = ccrs.LambertAzimuthalEqualArea(central_longitude=-105,central_latitude=74)

# Coordinates to transform (lon, lat) in the Plate Carree projection
geo_extent = [-140, -70, 64, 79]

def transform_coords(lon, lat, projection):
    x, y = projection.transform_point(lon, lat, ccrs.PlateCarree())
    return x, y

corners = [
    (geo_extent[0], geo_extent[2]),  # SW corner
    (geo_extent[1], geo_extent[2]),  # SE corner
    (geo_extent[0], geo_extent[3]),  # NW corner
    (geo_extent[1], geo_extent[3])   # NE corner
]

# Transform coordinates to the desired map area
proj_corners = [transform_coords(lon, lat, map_projection) for lon, lat in corners]
x_coords, y_coords = zip(*proj_corners)

x_min, x_max = min(x_coords), max(x_coords)
y_min, y_max = min(y_coords), max(y_coords)

grid_spacing = 25000 # 25 km

x_grid_centre = np.arange(x_min-grid_spacing/2,x_max + grid_spacing, grid_spacing)
y_grid_centre = np.arange(y_min-grid_spacing/2,y_max + grid_spacing, grid_spacing)
new_x_centre, new_y_centre = np.meshgrid(x_grid_centre, y_grid_centre)

x_grid_edge = np.arange(x_min,x_max + grid_spacing, grid_spacing)
y_grid_edge = np.arange(y_min,y_max + grid_spacing, grid_spacing)
new_x_edge, new_y_edge= np.meshgrid(x_grid_edge, y_grid_edge)

print('Edges grid shape = ',np.shape(new_x_edge))
print('Centre grid shape = ',np.shape(new_x_centre))


# # # # 
# Griding Files
# Navigate to CS2 Files

grid_output_folder = ('/home/amyswiggs/Chapter2/NWP_CS2_Grids_Cartopy/')

#years = ['2011','2012','2013','2014','2015','2016','2017','2018','2019','2020','2021','2022']
years = ['2023']
months = ['01','02','03','04','05','09','10','11','12']
#years = ['2010']
#months = ['10','11','12']

# Loop through years and months
for year in years:
    for month in months:
        
        CS2_files = sorted(glob.glob('/cpdata/SATS/RA/CRY/processed/arco/'+str(year)+str(month)+'_MERGE/*.elev'))
        
        lons,lats,types,peakiness,iceconc,ssd =[],[],[],[],[],[]
        
        # Loop through files
        for file in CS2_files:
            try:
                print(file)
                data = T(gftxt((file)))

                lon = data[6]
                lat = data[5]
                typ = data[0]
                icep = data[11]

                lons.extend(lon)
                lats.extend(lat)
                types.extend(typ)
                iceconc.extend(icep)

                ice_conc = np.array(iceconc)
                conc_0 = np.where(ice_conc < 0, np.nan, ice_conc)

                mask = np.isnan(conc_0)

                types_0 = np.where(mask, np.nan, types)

                ocean_val = 1
                ocean_values = np.where(types_0 == ocean_val, 1,0)

                floe_val = 3
                floe_values = np.where(types_0 == floe_val,1,0)

                lead_val = 2
                lead_values = np.where(types_0 == lead_val,1,0)


            except Exception as e:
                print(f"An error occurred while processing {file}: {e}")
                continue


        lons_array = np.array(lons)
        lats_array = np.array(lats)
        
        # Grid the data 
        x, y = map_projection.transform_points(ccrs.PlateCarree(), lons_array, lats_array)[:, :2].T
        
        # Set minimum count, and count the number of values
        min_scipy_count = T(scipy.stats.binned_statistic_2d(x,y,types_0,bins=(x_grid_edge,y_grid_edge),statistic='count')[0])

        #
        count_ocean=T(scipy.stats.binned_statistic_2d(x,y,ocean_values,bins=(x_grid_edge,y_grid_edge),statistic='sum')[0])
        count_min_ocean = np.where(min_scipy_count > 50, count_ocean,np.nan)
        np.save(grid_output_folder+str(year)+str(month)+'_CountOceanRegrid.npy',count_min_ocean)
        
        #
        count_floes=T(scipy.stats.binned_statistic_2d(x,y,floe_values,bins=(x_grid_edge,y_grid_edge),statistic='sum')[0])
        count_min_floes = np.where(min_scipy_count > 50, count_floes,np.nan)
        np.save(grid_output_folder+str(year)+str(month)+'_CountFloesRegrid.npy',count_min_floes)
        
        #
        count_leads=T(scipy.stats.binned_statistic_2d(x,y,lead_values,bins=(x_grid_edge,y_grid_edge),statistic='sum')[0])
        count_min_leads = np.where(min_scipy_count > 50, count_leads,np.nan)
        np.save(grid_output_folder+str(year)+str(month)+'_CountLeadsRegrid.npy',count_min_leads)
        
        # 
        conc0_binned = T(scipy.stats.binned_statistic_2d(x,y,conc_0,bins=(x_grid_edge,y_grid_edge),statistic=np.nanmean)[0])
        conc0_min_binned = np.where(min_scipy_count > 50, conc0_binned,np.nan)
        np.save(grid_output_folder+str(year)+str(month)+'_ConcentrationRegrid.npy',conc0_min_binned)